<a href="https://colab.research.google.com/github/Gireesheruvaram/LLM-Beginer/blob/LLM-advanced-kata1/LLM_Adv_kata1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective : Movie Recommendation with User Preferences**

In [ ]:
!pip install pandas numpy

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# File path
file_path = "/content/drive/MyDrive/LangChain/movie_ratings.xlsx"

# Error handling for loading the dataset
try:
    df = pd.read_excel(file_path)
except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found.")
    exit()
except Exception as e:
    print(f"Error loading the dataset: {e}")
    exit()

In [ ]:
# Error handling for genre parsing
try:
    # Parse genres from stringified lists
    df["genres"] = df["genres"].apply(lambda x: eval(x) if isinstance(x, str) else x)
except Exception as e:
    print(f"Error parsing genres: {e}")
    exit()

# Print unique user IDs
unique_users = df["userId"].unique()
print(f"\nUnique User IDs in DataFrame: {unique_users}")

# Define unique genres
unique_genres = set(genre for genres in df["genres"] for genre in genres)
print(f"\nUnique Genres in DataFrame: {unique_genres}")


Unique User IDs in DataFrame: ['u1' 'u2' 'u3' 'u4' 'u5' 'u6' 'u7' 'u8' 'u9' 'u10' 'u11' 'u12' 'u13'
 'u14' 'u15']

Unique Genres in DataFrame: {'Thriller', 'Drama', 'Adventure', 'Crime', 'War', 'Comedy', 'Romance', 'Fantasy', 'History', 'Sci-Fi', 'Action'}


In [ ]:
# Initialize user preference vectors
user_preferences = {user: np.zeros(len(unique_genres)) for user in unique_users}
genre_index = {genre: i for i, genre in enumerate(unique_genres)}

# Update user preference vectors based on ratings
for _, row in df.iterrows():
    try:
        user = row["userId"]
        rating = row["rating"]
        genres = row["genres"]
        if user not in user_preferences:
            user_preferences[user] = np.zeros(len(unique_genres))
        for genre in genres:
            if genre in genre_index:
                user_preferences[user][genre_index[genre]] += rating
    except Exception as e:
        print(f"Error processing row {row}: {e}")
        continue

# Print user preference vectors
print("\nUser Preference Vectors:")
for user, vector in user_preferences.items():
    print(f"User {user}: {vector}")


User Preference Vectors:
User u1: [ 0.  9.  0. 12.  0.  0.  0.  0.  0.  0.  3.]
User u2: [0. 0. 4. 0. 0. 0. 0. 4. 0. 5. 5.]
User u3: [0. 0. 5. 4. 0. 4. 0. 0. 0. 5. 5.]
User u4: [0. 9. 0. 5. 0. 0. 4. 5. 0. 0. 0.]
User u5: [4. 5. 0. 4. 0. 0. 0. 0. 0. 0. 0.]
User u6: [0. 9. 0. 0. 4. 0. 0. 0. 5. 0. 0.]
User u7: [0. 9. 0. 9. 0. 0. 0. 0. 0. 0. 0.]
User u8: [0. 0. 4. 5. 0. 0. 0. 4. 0. 0. 5.]
User u9: [0. 0. 4. 0. 4. 0. 0. 0. 0. 5. 5.]
User u10: [0. 4. 0. 9. 0. 5. 0. 0. 0. 0. 0.]
User u11: [5. 9. 0. 5. 0. 0. 4. 0. 0. 0. 0.]
User u12: [4. 9. 0. 5. 0. 0. 0. 5. 0. 0. 0.]
User u13: [0. 9. 0. 0. 4. 0. 0. 0. 5. 0. 0.]
User u14: [0. 9. 0. 9. 0. 0. 0. 0. 0. 0. 0.]
User u15: [0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 5.]


**Recommendation Funtion**

In [ ]:
# Recommendation Function with Error Handling
def recommend_movies(user_id, user_preferences, external_movie_list, genre_index, top_n=3):
    try:
        if user_id not in user_preferences:
            raise ValueError(f"User {user_id} not found in user preferences.")

        user_vector = user_preferences[user_id]
        recommendations = []

        for movie in external_movie_list:
            try:
                # Validate the movie structure
                if "Title" not in movie or "Genre Vector" not in movie:
                    raise KeyError("Missing 'Title' or 'Genre Vector' in movie data.")

                genre_vector = np.zeros(len(unique_genres))

                # For each genre in the movie, mark the corresponding genre position in genre_vector
                for genre in movie["Genre Vector"]:
                    if genre in genre_index:
                        genre_vector[genre_index[genre]] = 1

                # Calculate the similarity score by taking the dot product between user preferences and genre vector
                similarity_score = np.dot(user_vector, genre_vector)
                recommendations.append((movie["Title"], similarity_score))
            except KeyError as e:
                print(f"Error in movie data: {e}")
                continue
            except Exception as e:
                print(f"Unexpected error with movie {movie['Title']}: {e}")
                continue

        # Sort recommendations by similarity score in descending order
        recommendations.sort(key=lambda x: x[1], reverse=True)
        return [title for title, _ in recommendations[:top_n]]

    except ValueError as e:
        print(f"Error: {e}")
        return []

In [ ]:
# Example External Movie List
external_movie_list = [
    {"Title": "The Godfather", "Genre Vector": ["Crime", "Drama"]},
    {"Title": "The Dark Knight", "Genre Vector": ["Action", "Crime"]},
    {"Title": "Inception", "Genre Vector": ["Action", "Sci-Fi"]},
    {"Title": "Forrest Gump", "Genre Vector": ["Drama", "Romance"]},
    {"Title": "The Lord of the Rings: The Fellowship of the Ring", "Genre Vector": ["Adventure", "Fantasy"]},
]

In [ ]:
# Print all available user IDs and select one
print(f"\nAvailable User IDs: {unique_users}")

# Choose a user ID for recommendations with error handling
try:
    user_id_to_recommend = input("Enter the User ID for which you want recommendations: ").strip()

    if user_id_to_recommend not in user_preferences:
        raise ValueError(f"User ID '{user_id_to_recommend}' not found in the dataset.")

    recommended_movies = recommend_movies(user_id_to_recommend, user_preferences, external_movie_list, genre_index)

    if recommended_movies:
        print(f"\nRecommended Movies for User {user_id_to_recommend}:")
        for title in recommended_movies:
            print(title)
    else:
        print(f"No recommendations found for User {user_id_to_recommend}.")

except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")



Available User IDs: ['u1' 'u2' 'u3' 'u4' 'u5' 'u6' 'u7' 'u8' 'u9' 'u10' 'u11' 'u12' 'u13'
 'u14' 'u15']
Enter the User ID for which you want recommendations: u3

Recommended Movies for User u3:
Inception
The Dark Knight
The Lord of the Rings: The Fellowship of the Ring
